In [50]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [51]:
# Download 3-month Treasury bill data and convert to monthly returns
tbill_monthly = yf.download("^IRX", start="2013-12-31", end="2023-12-31").resample('ME').last()['Close'] / 100 / 12
tbill_monthly = tbill_monthly.iloc[1:]  # Align with etf_returns (drop first price, keep returns)

[*********************100%***********************]  1 of 1 completed


In [52]:
# Download ETF data and convert to monthly returns (auto_adjust=True accounts for dividends)
etf_returns = (yf.download(["SPY", "EEM", "FM"], start="2013-12-31", end="2023-12-31", auto_adjust=True)
               .xs("Close", level="Price", axis=1)
               .resample("ME").last()
               .pct_change()
               .dropna())


[*********************100%***********************]  3 of 3 completed


In [53]:
# Print shapes for all dataframes/series
print("tbill_monthly shape:", tbill_monthly.shape)
print("etf_returns shape:", etf_returns.shape)


tbill_monthly shape: (120, 1)
etf_returns shape: (120, 3)


In [54]:
print("tbill_monthly head:", tbill_monthly.head())
print("etf_returns head:", etf_returns.head())

tbill_monthly head: Ticker          ^IRX
Date                
2014-01-31  0.000008
2014-02-28  0.000036
2014-03-31  0.000023
2014-04-30  0.000017
2014-05-31  0.000025
etf_returns head: Ticker           EEM        FM       SPY
Date                                    
2014-01-31 -0.086363  0.001778 -0.035248
2014-02-28  0.033778  0.033432  0.045515
2014-03-31  0.038754  0.037790  0.008295
2014-04-30  0.007803  0.025655  0.006952
2014-05-31  0.029518  0.045992  0.023206


In [ ]:
# Calculate risk-adjusted return metrics for each ETF
tbill_aligned = tbill_monthly.squeeze().reindex(etf_returns.index)
excess_returns = etf_returns.sub(tbill_aligned, axis=0)

# Calculate metrics
mean_annual = etf_returns.mean() * 12
vol_annual = etf_returns.std() * np.sqrt(12)
mean_excess_annual = excess_returns.mean() * 12
sharpe = mean_excess_annual / vol_annual

# Sortino ratio: downside deviation of negative excess returns
downside_dev = excess_returns[excess_returns < 0].std() * np.sqrt(12)
sortino = mean_excess_annual / downside_dev

# Create results table
results_table = pd.DataFrame({
    'Mean Annualized Return': mean_annual,
    'Annualized Volatility': vol_annual,
    'Sharpe Ratio': sharpe,
    'Sortino Ratio': sortino
}).round(4)

print("Risk-Adjusted Return Metrics:")
print("=" * 60)
print(results_table)